In [1]:
import pandas as pd

In [8]:
csv = "./data/eightthirtyfour.com/2000-01_pbp.csv"

In [9]:
data = pd.read_csv(csv)

In [10]:
data.dtypes

Unnamed: 0                     int64
EVENTMSGACTIONTYPE             int64
EVENTMSGTYPE                   int64
EVENTNUM                       int64
GAME_ID                        int64
HOMEDESCRIPTION               object
NEUTRALDESCRIPTION           float64
PCTIMESTRING                  object
PERIOD                         int64
PERSON1TYPE                  float64
PERSON2TYPE                    int64
PERSON3TYPE                    int64
PLAYER1_ID                     int64
PLAYER1_NAME                  object
PLAYER1_TEAM_ABBREVIATION     object
PLAYER1_TEAM_CITY             object
PLAYER1_TEAM_ID              float64
PLAYER1_TEAM_NICKNAME         object
PLAYER2_ID                     int64
PLAYER2_NAME                  object
PLAYER2_TEAM_ABBREVIATION     object
PLAYER2_TEAM_CITY             object
PLAYER2_TEAM_ID              float64
PLAYER2_TEAM_NICKNAME         object
PLAYER3_ID                     int64
PLAYER3_NAME                  object
PLAYER3_TEAM_ABBREVIATION     object
P

In [14]:
data.head(100)

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
0,0,0,12,0,20001116,NaN,NaN,12:00,1,0.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:11 PM
1,1,0,10,1,20001116,Jump Ball Grant vs. Mutombo: Tip to Hill,NaN,12:00,1,4.0,...,238,Tyrone Hill,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,NaN,12:12 PM
2,2,1,1,2,20001116,NaN,NaN,11:45,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Iverson 21' Jump Shot (2 PTS),12:13 PM
3,3,1,6,3,20001116,NaN,NaN,11:28,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hill P.FOUL (P1.T1),12:13 PM
4,4,1,1,4,20001116,Mason 17' Jump Shot (2 PTS),NaN,11:19,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,2 - 2,TIE,NaN,12:13 PM
5,5,1,2,5,20001116,NaN,NaN,10:56,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Lynch 22' Jump Shot,12:14 PM
6,6,0,4,6,20001116,Mason REBOUND (Off:0 Def:1),NaN,10:54,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:14 PM
7,7,42,2,7,20001116,MISS Majerle Driving Layup,NaN,10:49,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:14 PM
8,8,0,4,8,20001116,NaN,NaN,10:47,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Snow REBOUND (Off:0 Def:1),12:14 PM
9,9,5,1,9,20001116,NaN,NaN,10:44,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,4 - 2,-2,Lynch Layup (2 PTS) (Snow 1 AST),12:14 PM


In [21]:
data['PERIOD'].unique()

array([1, 2, 3, 4, 5, 6, 7])